# 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd
import json

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

from collections import defaultdict
from itertools import chain, combinations
from optparse import OptionParser

In [2]:
!pip install mlxtend

In [3]:
?apriori

In [4]:
sample = [["사과", "맥주", "치킨", "쌀"], ["사과", "맥주", "쌀"], ["사과", "맥주"], 
          ["사과", "바나나"], ["맥주", "치킨", "우유", "쌀"], ["맥주", "우유", "쌀"],
          ["맥주", "우유"], ["사과", "바나나"]]

In [5]:
te = TransactionEncoder()
te_result = te.fit(sample).transform(sample)

In [6]:
te_result

array([[ True, False,  True,  True, False,  True],
       [ True, False,  True,  True, False, False],
       [ True, False,  True, False, False, False],
       [False,  True,  True, False, False, False],
       [ True, False, False,  True,  True,  True],
       [ True, False, False,  True,  True, False],
       [ True, False, False, False,  True, False],
       [False,  True,  True, False, False, False]])

In [7]:
sample_df = pd.DataFrame(te_result, columns=["사과", "바나나", "맥주", "치킨", "우유", "쌀"])

In [8]:
sample_df

,사과,바나나,맥주,치킨,우유,쌀
0,True,False,True,True,False,True
1,True,False,True,True,False,False
2,True,False,True,False,False,False
3,False,True,True,False,False,False
4,True,False,False,True,True,True
5,True,False,False,True,True,False
6,True,False,False,False,True,False
7,False,True,True,False,False,False


In [9]:
itemsets = apriori(sample_df, min_support=0.5, use_colnames=True)
itemsets

,support,itemsets
0,0.750,(사과)
1,0.625,(맥주)
2,0.500,(치킨)
3,0.500,"(사과, 치킨)"


In [10]:
association_rules(itemsets, metric="confidence", min_threshold=0.1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(사과),(치킨),0.75,0.50,0.5,0.666667,1.333333,0.125,1.5
1,(치킨),(사과),0.50,0.75,0.5,1.000000,1.333333,0.125,inf


# 레시피 json 파일 열기

In [11]:
with open('Final/Deduplicated_Filtered_recipe_data_Final.json', encoding="utf-8") as f:
    data = json.load(f)

# 데이터 확인

In [12]:
data

{'data': [{'code': 0,
   'title': '맛살볶음밥',
   'ingredients': ['맛살',
    '맛술',
    '후춧가루',
    '간장',
    '즉석밥',
    '소금',
    '다진마늘',
    '멸치액젓',
    '대파',
    '설탕',
    '계란'],
   'url': 'https://2bob.co.kr/recipe.php?id=view&eTheme=1&idx=1750'},
  {'code': 1,
   'title': '돈가스 김밥 샌드위치',
   'ingredients': ['청피망',
    '당근',
    '김',
    '단무지',
    '즉석밥',
    '소금',
    '참기름',
    '깻잎',
    '돈가스소스',
    '돈가스'],
   'url': 'https://2bob.co.kr/recipe.php?id=view&eTheme=1&idx=1625'},
  {'code': 2,
   'title': '묵은지말이 참치김밥',
   'ingredients': ['참깨', '즉석밥', '묵은지', '마요네즈', '소금', '통조림참치', '참기름'],
   'url': 'https://2bob.co.kr/recipe.php?id=view&eTheme=1&idx=1628'},
  {'code': 3,
   'title': '마늘종장아찌 볶음밥',
   'ingredients': ['후춧가루', '즉석밥', '마늘', '참기름', '스팸', '마늘종장아찌', '계란'],
   'url': 'https://2bob.co.kr/recipe.php?id=view&eTheme=1&idx=1754'},
  {'code': 4,
   'title': '새우볶음밥',
   'ingredients': ['냉동새우', '후춧가루', '간장', '즉석밥', '소금', '마늘', '대파', '계란'],
   'url': 'https://2bob.co.kr/recipe.php?id=view&eTh

In [13]:
df_raw = pd.DataFrame(data["data"])

In [14]:
df_raw

,code,title,ingredients,url
0,0,맛살볶음밥,"[맛살, 맛술, 후춧가루, 간장, 즉석밥, 소금, 다진마늘, 멸치액젓, 대파, 설탕...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
1,1,돈가스 김밥 샌드위치,"[청피망, 당근, 김, 단무지, 즉석밥, 소금, 참기름, 깻잎, 돈가스소스, 돈가스]",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
2,2,묵은지말이 참치김밥,"[참깨, 즉석밥, 묵은지, 마요네즈, 소금, 통조림참치, 참기름]",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
3,3,마늘종장아찌 볶음밥,"[후춧가루, 즉석밥, 마늘, 참기름, 스팸, 마늘종장아찌, 계란]",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
4,4,새우볶음밥,"[냉동새우, 후춧가루, 간장, 즉석밥, 소금, 마늘, 대파, 계란]",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
...,...,...,...,...
1665,2908,퓨전라따뚜이,"[토마토소스, 파라타, 방울토마토, 바질가루, 주키니호박, 올리브유, 가지, 노란파...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
1666,2910,누룽지 카레 그라탱,"[누룽지, 크림수프가루, 카레가루, 슈레드모차렐라치즈, 파슬리]",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
1667,2911,감자베이컨스튜,"[감자, 닭육수, 양파, 베이컨, 파슬리, 타임, 쪽파, 셀러리, 대파, 백태, 양배추]",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
1668,2912,싸 먹는 비빔떡,"[참깨, 고추장, 계란, 식초, 당근, 간장, 소금, 무, 물엿, 밀가루, 다진마늘...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...


# 사용자가 입력한 재료가 포함된 레시피만 가져오기

In [15]:
user = ["소금", "후춧가루"]

In [16]:
user_lst = []
for i in range(len(df_raw)):
    count = 0
    for j in user:
        if(j in df_raw.iloc[i]["ingredients"]):
            count = count + 1
        
        if count == len(user) :
            user_lst.append(tuple(df_raw.iloc[i]))

In [17]:
user_df = pd.DataFrame(user_lst, columns=["code", "title", "ingredients", "url"])
user_df

,code,title,ingredients,url
0,0,맛살볶음밥,"[맛살, 맛술, 후춧가루, 간장, 즉석밥, 소금, 다진마늘, 멸치액젓, 대파, 설탕...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
1,4,새우볶음밥,"[냉동새우, 후춧가루, 간장, 즉석밥, 소금, 마늘, 대파, 계란]",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
2,14,크림볶음밥,"[맛술, 생강, 양파, 후춧가루, 간장, 즉석밥, 슈레드모차렐라치즈, 소금, 참기름...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
3,20,소고기 오이볶음밥,"[후춧가루, 간장, 즉석밥, 홍고추, 소금, 참기름, 다진마늘, 다진소고기, 오이,...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
4,25,카르보나라 덮밥,"[양파, 베이컨, 후춧가루, 즉석밥, 소금, 슬라이스체다치즈, 다진마늘, 버터, 생...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
...,...,...,...,...
417,2882,세발나물달걀전,"[맛술, 당근, 후춧가루, 세발나물, 소금, 통조림참치, 다진마늘, 대파, 계란]",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
418,2883,쑥갓두부전,"[식초, 양파, 후춧가루, 느타리버섯, 간장, 쑥갓, 홍고추, 소금, 생수, 부침가...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
419,2896,싸 먹는 타코,"[스위트칠리소스, 후춧가루, 토르티야, 간장, 빨간파프리카, 청주, 소금, 양상추,...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
420,2906,프렌치어니언수프,"[바게트, 양파, 후춧가루, 박력분, 소금, 슈레드에멘탈치즈, 후추, 버터, 레드와...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...


In [18]:
user_df.to_excel('user_df.xlsx')

In [19]:
df = pd.read_excel('user_df.xlsx')

In [20]:
df = df.drop(["Unnamed: 0"], axis=1)

In [21]:
df = df.drop_duplicates()
df

,code,title,ingredients,url
0,0,맛살볶음밥,"['맛살', '맛술', '후춧가루', '간장', '즉석밥', '소금', '다진마늘'...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
1,4,새우볶음밥,"['냉동새우', '후춧가루', '간장', '즉석밥', '소금', '마늘', '대파'...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
2,14,크림볶음밥,"['맛술', '생강', '양파', '후춧가루', '간장', '즉석밥', '슈레드모차...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
3,20,소고기 오이볶음밥,"['후춧가루', '간장', '즉석밥', '홍고추', '소금', '참기름', '다진마...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
4,25,카르보나라 덮밥,"['양파', '베이컨', '후춧가루', '즉석밥', '소금', '슬라이스체다치즈',...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
...,...,...,...,...
417,2882,세발나물달걀전,"['맛술', '당근', '후춧가루', '세발나물', '소금', '통조림참치', '다...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
418,2883,쑥갓두부전,"['식초', '양파', '후춧가루', '느타리버섯', '간장', '쑥갓', '홍고추...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
419,2896,싸 먹는 타코,"['스위트칠리소스', '후춧가루', '토르티야', '간장', '빨간파프리카', '청...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
420,2906,프렌치어니언수프,"['바게트', '양파', '후춧가루', '박력분', '소금', '슈레드에멘탈치즈',...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...


In [22]:
df_raw = df.reset_index(drop=True)
df_raw

,code,title,ingredients,url
0,0,맛살볶음밥,"['맛살', '맛술', '후춧가루', '간장', '즉석밥', '소금', '다진마늘'...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
1,4,새우볶음밥,"['냉동새우', '후춧가루', '간장', '즉석밥', '소금', '마늘', '대파'...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
2,14,크림볶음밥,"['맛술', '생강', '양파', '후춧가루', '간장', '즉석밥', '슈레드모차...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
3,20,소고기 오이볶음밥,"['후춧가루', '간장', '즉석밥', '홍고추', '소금', '참기름', '다진마...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
4,25,카르보나라 덮밥,"['양파', '베이컨', '후춧가루', '즉석밥', '소금', '슬라이스체다치즈',...",https://2bob.co.kr/recipe.php?id=view&eTheme=1...
...,...,...,...,...
417,2882,세발나물달걀전,"['맛술', '당근', '후춧가루', '세발나물', '소금', '통조림참치', '다...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
418,2883,쑥갓두부전,"['식초', '양파', '후춧가루', '느타리버섯', '간장', '쑥갓', '홍고추...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
419,2896,싸 먹는 타코,"['스위트칠리소스', '후춧가루', '토르티야', '간장', '빨간파프리카', '청...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...
420,2906,프렌치어니언수프,"['바게트', '양파', '후춧가루', '박력분', '소금', '슈레드에멘탈치즈',...",https://2bob.co.kr/recipe.php?id=view&eTheme=2...


# 재료 원핫인코딩

In [23]:
df_raw["ingredients"]

0      ['맛살', '맛술', '후춧가루', '간장', '즉석밥', '소금', '다진마늘'...
1      ['냉동새우', '후춧가루', '간장', '즉석밥', '소금', '마늘', '대파'...
2      ['맛술', '생강', '양파', '후춧가루', '간장', '즉석밥', '슈레드모차...
3      ['후춧가루', '간장', '즉석밥', '홍고추', '소금', '참기름', '다진마...
4      ['양파', '베이컨', '후춧가루', '즉석밥', '소금', '슬라이스체다치즈',...
                             ...                        
417    ['맛술', '당근', '후춧가루', '세발나물', '소금', '통조림참치', '다...
418    ['식초', '양파', '후춧가루', '느타리버섯', '간장', '쑥갓', '홍고추...
419    ['스위트칠리소스', '후춧가루', '토르티야', '간장', '빨간파프리카', '청...
420    ['바게트', '양파', '후춧가루', '박력분', '소금', '슈레드에멘탈치즈',...
421    ['감자', '바게트', '후춧가루', '소금', '마늘', '올리브유', '우유'...
Name: ingredients, Length: 422, dtype: object

In [24]:
type(df_raw["ingredients"][1])

str

### 재료가 문자열 형태, 오류 발생

In [25]:
import ast
list_in_str = df_raw["ingredients"][1]
list_data = ast.literal_eval(list_in_str)
print(type(list_data))

<class 'list'>


In [26]:
list_data

['냉동새우', '후춧가루', '간장', '즉석밥', '소금', '마늘', '대파', '계란']

In [27]:
list_data = []
for i in range(len(df_raw["ingredients"])):    
    list_data.append(ast.literal_eval(df_raw["ingredients"][i]))

In [28]:
type(list_data)

list

In [29]:
data_en = []
for i in list_data:
    data_en.append(i)

In [30]:
data_en

[['맛살', '맛술', '후춧가루', '간장', '즉석밥', '소금', '다진마늘', '멸치액젓', '대파', '설탕', '계란'],
 ['냉동새우', '후춧가루', '간장', '즉석밥', '소금', '마늘', '대파', '계란'],
 ['맛술',
  '생강',
  '양파',
  '후춧가루',
  '간장',
  '즉석밥',
  '슈레드모차렐라치즈',
  '소금',
  '참기름',
  '청양고추',
  '다진마늘',
  '다진소고기',
  '우유',
  '대파',
  '생크림',
  '설탕'],
 ['후춧가루', '간장', '즉석밥', '홍고추', '소금', '참기름', '다진마늘', '다진소고기', '오이', '굴소스', '설탕'],
 ['양파',
  '베이컨',
  '후춧가루',
  '즉석밥',
  '소금',
  '슬라이스체다치즈',
  '다진마늘',
  '버터',
  '생크림',
  '청피망',
  '파슬리'],
 ['양파',
  '후춧가루',
  '참깨',
  '소금',
  '참기름',
  '고추장',
  '애호박',
  '표고버섯',
  '다진소고기',
  '올리브유',
  '쌀',
  '토마토',
  '올리고당'],
 ['삼치', '후춧가루', '은행', '간장', '청주', '소금', '쪽파', '레몬즙', '쌀', '설탕'],
 ['당근', '양파', '후춧가루', '즉석밥', '소금', '참기름', '애호박', '통조림닭가슴살'],
 ['양파', '후춧가루', '간장', '즉석밥', '매실청', '소금', '소고기안심', '참기름', '고추냉이', '버터', '계란'],
 ['후춧가루', '즉석밥', '연두부', '소금', '참기름', '슬라이스체다치즈', '버터', '계란'],
 ['맛술',
  '국간장',
  '후춧가루',
  '간장',
  '참깨',
  '즉석밥',
  '소금',
  '참기름',
  '다진마늘',
  '다진소고기',
  '열무',
  '대파',
  '설탕'],
 ['당근', '양파', '후춧가루', '통조림골뱅이', '들깻

In [31]:
te = TransactionEncoder()
te_ary = te.fit(data_en).transform(data_en)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [32]:
df

,가다랑어포,가람마살라,가래떡,가쓰오육수,가지,간장,갈색설탕,갈치,감자,감자칩,...,화자오,황금송이버섯,황기,황설탕,황태머리,후추,후춧가루,훈제오리,흑맥주,흑설탕
0,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
418,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
419,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
420,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False


# support 탐색

In [33]:
frequent = apriori(df, min_support=0.1, use_colnames=True, verbose=True)
frequent
frequent.sort_values("support", ascending=False)

Processing 80 combinations | Sampling itemset size 5


,support,itemsets
14,1.000000,(소금)
75,1.000000,"(후춧가루, 소금)"
22,1.000000,(후춧가루)
69,0.438389,"(소금, 양파)"
78,0.438389,"(후춧가루, 양파)"
...,...,...
149,0.101896,"(후춧가루, 다진마늘, 대파, 간장)"
162,0.101896,"(다진마늘, 소금, 대파, 참기름)"
35,0.101896,"(후춧가루, 고추장)"
2,0.101896,(고추장)


In [34]:
# pd.set_option('display.max_rows', 10000)

In [35]:
features = df.astype(float)
features

,가다랑어포,가람마살라,가래떡,가쓰오육수,가지,간장,갈색설탕,갈치,감자,감자칩,...,화자오,황금송이버섯,황기,황설탕,황태머리,후추,후춧가루,훈제오리,흑맥주,흑설탕
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
418,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
419,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


# 레시피 제목 추출

In [36]:
labels = df_raw["title"]
labels

0            맛살볶음밥
1            새우볶음밥
2            크림볶음밥
3        소고기 오이볶음밥
4         카르보나라 덮밥
          ...     
417        세발나물달걀전
418          쑥갓두부전
419        싸 먹는 타코
420       프렌치어니언수프
421    마늘크림수프와 마늘빵
Name: title, Length: 422, dtype: object

In [37]:
### test
# lst = []
# for i in labels:
#     lst.append(i)
    
# lst

# Apriori Algorithm

In [38]:
def powerset(s): # 멱집합
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))


def getAboveMinSup(itemSet, itemSetList, minSup, globalItemSetWithSup):
    freqItemSet = set()
    localItemSetWithSup = defaultdict(int)

    for item in itemSet:
        for itemSet in itemSetList:
            if item.issubset(itemSet):
                globalItemSetWithSup[item] += 1
                localItemSetWithSup[item] += 1

    for item, supCount in localItemSetWithSup.items():
        support = float(supCount / len(itemSetList))
        if(support >= minSup):
            freqItemSet.add(item)

    return freqItemSet


def getUnion(itemSet, length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def pruning(candidateSet, prevFreqSet, length):
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if(frozenset(subset) not in prevFreqSet):
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet


def associationRule(freqItemSet, recipes, labels, itemSetWithSup, minConf):
    rules = []
    lbls = []
    
    sets_compare = [set(re) for re in recipes]
    print(sets_compare)
    for k, itemSet in freqItemSet.items():
        for item in itemSet:
            subsets = powerset(item)
            
            for i in labels:
                lbls.append(i)
            
            for s in subsets:
                ss = set([es for es in s])
                if(len(ss.intersection(sets_compare[0])) > 0 and len(ss.intersection(sets_compare[1])) > 0):
                    lbls.append([labels[0], labels[1]])
                if(len(ss.intersection(sets_compare[0])) > 0):
                    lbls.append(labels[0])
                if(len(ss.intersection(sets_compare[1])) > 0):
                    lbls.append(labels[1])                    
                    
                confidence = float(
                    itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if(confidence > minConf):
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules, lbls


def getItemSetFromList(itemSetList):
    tempItemSet = set()

    for itemSet in itemSetList:
        for item in itemSet:
            tempItemSet.add(frozenset([item]))

    return tempItemSet

In [39]:
def apriori(itemSetList, labels, minSup, minConf):
    C1ItemSet = getItemSetFromList(itemSetList)
    # Final result global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(
        C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(
            candidateSet, itemSetList, minSup, globalItemSetWithSup)
        k += 1
        
    rules, lbls = associationRule(globalFreqItemSet, itemSetList, labels, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules, lbls

In [40]:
idx_to_column = {k:v for k,v in enumerate(features.columns.values)}
column_to_idx = {v:k for k,v in enumerate(features.columns.values)}

In [41]:
def get_recipes_with_ingredients(features, labels):
    idxs = np.array([np.arange(len(features.columns.values)).tolist() for x in range(len(labels))])
    multiply = features.iloc[:].values.astype(np.int32) * idxs
    idx_arr  = [np.where(mult != 0)[0].tolist() for mult in multiply]
    recipes = []
    for arr_idx in idx_arr:
        temp_result = []
        for idx in arr_idx:
            temp_result.append(idx_to_column[idx])
        recipes.append(temp_result)
    return recipes, labels[:].values

In [42]:
def print_recipe(recipe, label):
    print("-" * 50)
    print("Recipe: ", label)
    print("-" * 50)
    for item in recipe:
        print(item)

In [43]:
# get recipes with certain ingredients
recipes, lbls = get_recipes_with_ingredients(features, labels)

# print recipes
pr = [print_recipe(rec, lbl) for rec, lbl in list(zip(recipes,lbls))]

--------------------------------------------------
Recipe:  맛살볶음밥
--------------------------------------------------
간장
계란
다진마늘
대파
맛살
맛술
멸치액젓
설탕
소금
즉석밥
후춧가루
--------------------------------------------------
Recipe:  새우볶음밥
--------------------------------------------------
간장
계란
냉동새우
대파
마늘
소금
즉석밥
후춧가루
--------------------------------------------------
Recipe:  크림볶음밥
--------------------------------------------------
간장
다진마늘
다진소고기
대파
맛술
생강
생크림
설탕
소금
슈레드모차렐라치즈
양파
우유
즉석밥
참기름
청양고추
후춧가루
--------------------------------------------------
Recipe:  소고기 오이볶음밥
--------------------------------------------------
간장
굴소스
다진마늘
다진소고기
설탕
소금
오이
즉석밥
참기름
홍고추
후춧가루
--------------------------------------------------
Recipe:  카르보나라 덮밥
--------------------------------------------------
다진마늘
버터
베이컨
생크림
소금
슬라이스체다치즈
양파
즉석밥
청피망
파슬리
후춧가루
--------------------------------------------------
Recipe:  토마토밥 & 토마토 약고추장
--------------------------------------------------
고추장
다진소고기
소금
쌀
애호박
양파
올리고당
올리브유
참기름
참깨
토마토
표고버섯
후춧가루


대파
두부
생강
생수
소금
양파
참기름
참깨
후춧가루
--------------------------------------------------
Recipe:  크림소스 캐비지롤 
--------------------------------------------------
다시마
다진돼지고기
빵가루
소금
양배추
양파
어린잎채소
우유
월계수잎
크림소스
후춧가루
--------------------------------------------------
Recipe:  해물찜
--------------------------------------------------
간장
고추장
고춧가루
꽃게
녹말
다진마늘
대파
미나리
미더덕
새우
생강
생수
설탕
소금
오징어
참기름
참치액젓
청주
콩나물
홍합
후춧가루
--------------------------------------------------
Recipe:  시금치달걀찜
--------------------------------------------------
계란
고춧가루
국간장
국물용멸치
다시마
다진마늘
맛술
새우젓
소금
시금치
참기름
후춧가루
--------------------------------------------------
Recipe:  토마토찜닭
--------------------------------------------------
간장
다진마늘
마늘
생닭
소금
양파
올리브유
청양고추
청주
토마토
후춧가루
--------------------------------------------------
Recipe:  에그인뽀빠이
--------------------------------------------------
계란
마늘
베이컨
소금
시금치
양송이버섯
양파
올리브유
토마토소스
파르메산치즈가루
프랑크소시지
할라피뇨피클
후춧가루
--------------------------------------------------
Recipe:  시금치 포테이토샐러드
-------------------------

In [44]:
freqItemSet, rules, lbls = apriori(recipes, lbls, minSup=0.25, minConf=0.25)

[{'멸치액젓', '간장', '설탕', '계란', '맛살', '소금', '맛술', '후춧가루', '대파', '다진마늘', '즉석밥'}, {'냉동새우', '간장', '마늘', '계란', '소금', '후춧가루', '대파', '즉석밥'}, {'생크림', '슈레드모차렐라치즈', '다진소고기', '간장', '설탕', '우유', '소금', '맛술', '후춧가루', '대파', '양파', '참기름', '청양고추', '다진마늘', '즉석밥', '생강'}, {'다진소고기', '간장', '설탕', '굴소스', '소금', '후춧가루', '참기름', '홍고추', '다진마늘', '즉석밥', '오이'}, {'생크림', '파슬리', '청피망', '슬라이스체다치즈', '소금', '후춧가루', '양파', '버터', '다진마늘', '즉석밥', '베이컨'}, {'참깨', '고추장', '다진소고기', '쌀', '소금', '올리고당', '토마토', '표고버섯', '올리브유', '후춧가루', '양파', '참기름', '애호박'}, {'쪽파', '삼치', '간장', '설탕', '쌀', '청주', '소금', '레몬즙', '후춧가루', '은행'}, {'통조림닭가슴살', '소금', '후춧가루', '양파', '당근', '참기름', '즉석밥', '애호박'}, {'간장', '계란', '고추냉이', '매실청', '소금', '소고기안심', '후춧가루', '양파', '버터', '참기름', '즉석밥'}, {'계란', '슬라이스체다치즈', '소금', '연두부', '후춧가루', '참기름', '버터', '즉석밥'}, {'참깨', '열무', '다진소고기', '간장', '설탕', '소금', '맛술', '후춧가루', '대파', '다진마늘', '참기름', '국간장', '즉석밥'}, {'통조림골뱅이', '소금', '들깻가루', '조미김', '후춧가루', '양파', '당근', '참기름', '즉석밥'}, {'참깨', '참치', '통조림옥수수', '마요네즈', '소금', '후춧가루', '참기름', '즉석밥', '오이'}, {'슈레드모차렐라치

In [45]:
freqItemSet

{1: {frozenset({'후춧가루'}),
  frozenset({'대파'}),
  frozenset({'양파'}),
  frozenset({'참기름'}),
  frozenset({'소금'}),
  frozenset({'간장'}),
  frozenset({'설탕'}),
  frozenset({'다진마늘'})},
 2: {frozenset({'간장', '후춧가루'}),
  frozenset({'다진마늘', '소금'}),
  frozenset({'소금', '후춧가루'}),
  frozenset({'소금', '양파'}),
  frozenset({'소금', '참기름'}),
  frozenset({'대파', '소금'}),
  frozenset({'간장', '소금'}),
  frozenset({'다진마늘', '후춧가루'}),
  frozenset({'설탕', '후춧가루'}),
  frozenset({'참기름', '후춧가루'}),
  frozenset({'대파', '후춧가루'}),
  frozenset({'양파', '후춧가루'}),
  frozenset({'설탕', '소금'})},
 3: {frozenset({'설탕', '소금', '후춧가루'}),
  frozenset({'다진마늘', '소금', '후춧가루'}),
  frozenset({'대파', '소금', '후춧가루'}),
  frozenset({'간장', '소금', '후춧가루'}),
  frozenset({'소금', '참기름', '후춧가루'}),
  frozenset({'소금', '양파', '후춧가루'})}}

In [46]:
rules

[[{'소금'}, {'참기름'}, 0.25829383886255924],
 [{'후춧가루'}, {'참기름'}, 0.25829383886255924],
 [{'후춧가루'}, {'소금', '참기름'}, 0.25829383886255924],
 [{'소금'}, {'참기름', '후춧가루'}, 0.25829383886255924],
 [{'소금', '후춧가루'}, {'참기름'}, 0.25829383886255924],
 [{'소금'}, {'대파'}, 0.26540284360189575],
 [{'후춧가루'}, {'대파'}, 0.26540284360189575],
 [{'후춧가루'}, {'대파', '소금'}, 0.26540284360189575],
 [{'소금'}, {'대파', '후춧가루'}, 0.26540284360189575],
 [{'소금', '후춧가루'}, {'대파'}, 0.26540284360189575],
 [{'후춧가루'}, {'설탕'}, 0.2796208530805687],
 [{'소금'}, {'설탕'}, 0.2796208530805687],
 [{'후춧가루'}, {'설탕', '소금'}, 0.2796208530805687],
 [{'소금'}, {'설탕', '후춧가루'}, 0.2796208530805687],
 [{'소금', '후춧가루'}, {'설탕'}, 0.2796208530805687],
 [{'후춧가루'}, {'간장'}, 0.35308056872037913],
 [{'소금'}, {'간장'}, 0.35308056872037913],
 [{'후춧가루'}, {'간장', '소금'}, 0.35308056872037913],
 [{'소금'}, {'간장', '후춧가루'}, 0.35308056872037913],
 [{'소금', '후춧가루'}, {'간장'}, 0.35308056872037913],
 [{'소금'}, {'다진마늘'}, 0.3933649289099526],
 [{'후춧가루'}, {'다진마늘'}, 0.3933649289099526],
 [{'후춧가루'}, 

In [47]:
lbls

['맛살볶음밥',
 '새우볶음밥',
 '크림볶음밥',
 '소고기 오이볶음밥',
 '카르보나라 덮밥',
 '토마토밥 & 토마토 약고추장',
 '삼치솥밥',
 '간편 닭죽',
 '탑스테이크 덮밥',
 '달걀덮밥',
 '소보로 열무밥',
 '골뱅이죽',
 '폭탄주먹밥',
 '삼각김밥누룽지전',
 '불고기 토핑밥',
 '새우젓 달걀볶음밥',
 '부추굴국밥',
 '김치볶음밥부리토',
 '데리야키고추오므라이스',
 '오믈렛밥',
 '얼갈이배추국밥',
 '된장돼지고기덮밥',
 '닭고기유부밥',
 '로제소스 리소토 ',
 '달걀말이밥',
 '아스파라거스 전복볶음밥',
 '옥수수김밥',
 '생와사비 참치마요김밥',
 '떠먹는 초밥 ',
 '중화풍 달걀볶음밥',
 '불고기장 쌈밥',
 '오므라이스',
 '캘리포니아롤',
 '돌솥비빔밥',
 '에비동',
 '유산슬덮밥',
 '불고기김초밥',
 '토마토소스 해물리소토',
 '카르보나라 라이스',
 '쇠고기김치주먹밥',
 '중화풍 호박고지덮밥',
 '테이크아웃 비빔밥',
 '시푸드데리컵밥',
 '버섯크림리소토',
 '알리오올리오 라이스',
 '4색덮밥',
 '김치볶음밥 돈가스롤',
 '소보로 오곡유부주머니',
 '스파이시닭구이 커리',
 '전복버터볶음밥',
 '약고추장곰취쌈밥',
 '데리야키마늘소스 채소덮밥',
 '매실장아찌볶음밥',
 '아스파라거스볶음밥',
 '모시조갯국',
 '돼지고기짬뽕국',
 '소고기 뭇국',
 '전복삼계탕',
 '백합 맑은탕',
 '봄동쇠고기국',
 '닭가슴살완자국',
 '마늘 조개탕',
 '한방오리백숙',
 '맑은 생태탕',
 '토마토누룽지수프',
 '갈비탕',
 '맑은 꽃게탕',
 '고사리 육개장',
 '5분 어묵국',
 '닭곰탕',
 '감자버섯수제비',
 '낙지불고기매운탕',
 '매콤한 쇠고기뭇국',
 '맑은 쇠고기뭇국',
 '얼큰홍합탕',
 '대파닭개장',
 '매운어묵배춧국',
 '삼계탕',
 '부추닭고기 물만두탕',
 '쇠고기우거지된장국',
 '경상도식 탕국',
 '쇠고기부추들깨탕',
 '감잣국',


In [48]:
recipes

[['간장', '계란', '다진마늘', '대파', '맛살', '맛술', '멸치액젓', '설탕', '소금', '즉석밥', '후춧가루'],
 ['간장', '계란', '냉동새우', '대파', '마늘', '소금', '즉석밥', '후춧가루'],
 ['간장',
  '다진마늘',
  '다진소고기',
  '대파',
  '맛술',
  '생강',
  '생크림',
  '설탕',
  '소금',
  '슈레드모차렐라치즈',
  '양파',
  '우유',
  '즉석밥',
  '참기름',
  '청양고추',
  '후춧가루'],
 ['간장', '굴소스', '다진마늘', '다진소고기', '설탕', '소금', '오이', '즉석밥', '참기름', '홍고추', '후춧가루'],
 ['다진마늘',
  '버터',
  '베이컨',
  '생크림',
  '소금',
  '슬라이스체다치즈',
  '양파',
  '즉석밥',
  '청피망',
  '파슬리',
  '후춧가루'],
 ['고추장',
  '다진소고기',
  '소금',
  '쌀',
  '애호박',
  '양파',
  '올리고당',
  '올리브유',
  '참기름',
  '참깨',
  '토마토',
  '표고버섯',
  '후춧가루'],
 ['간장', '레몬즙', '삼치', '설탕', '소금', '쌀', '은행', '쪽파', '청주', '후춧가루'],
 ['당근', '소금', '애호박', '양파', '즉석밥', '참기름', '통조림닭가슴살', '후춧가루'],
 ['간장', '계란', '고추냉이', '매실청', '버터', '소고기안심', '소금', '양파', '즉석밥', '참기름', '후춧가루'],
 ['계란', '버터', '소금', '슬라이스체다치즈', '연두부', '즉석밥', '참기름', '후춧가루'],
 ['간장',
  '국간장',
  '다진마늘',
  '다진소고기',
  '대파',
  '맛술',
  '설탕',
  '소금',
  '열무',
  '즉석밥',
  '참기름',
  '참깨',
  '후춧가루'],
 ['당근', '들깻가루', '소금', '양파', '조미김', 

In [49]:
def find_recipe(ingredients, rules, lbls):
    
    user_ing = set(ingredients)
    additional_ing = set()
    for i in rules:
        if (user_ing == i[0]):
            for j in i[1]:
                additional_ing.add(j)

    additional = list(additional_ing)
    
    add_recipe = []
    
    for i in additional:
        add_ing = []
        add_ing = list(user_ing)
        add_ing.append(i)
        
        for j, re in enumerate(recipes):
            if (add_ing[0] in re and add_ing[1] in re and add_ing[2]):
                add_recipe.append(j)
                
    add_recipe = set(add_recipe)
    
    recipe_name = []
    
    for i in add_recipe :
        recipe_name.append(lbls[i])
        
    return recipe_name

In [50]:
print('{0}과 {1}를 재료로 사용하는 레시피 찾는 중...'.format("소금", "후춧가루"))
found_recipe = find_recipe(user, rules, lbls)
print('찾은 레시피: ', found_recipe)

소금과 후춧가루를 재료로 사용하는 레시피 찾는 중...
찾은 레시피:  ['맛살볶음밥', '새우볶음밥', '크림볶음밥', '소고기 오이볶음밥', '카르보나라 덮밥', '토마토밥 & 토마토 약고추장', '삼치솥밥', '간편 닭죽', '탑스테이크 덮밥', '달걀덮밥', '소보로 열무밥', '골뱅이죽', '폭탄주먹밥', '삼각김밥누룽지전', '불고기 토핑밥', '새우젓 달걀볶음밥', '부추굴국밥', '김치볶음밥부리토', '데리야키고추오므라이스', '오믈렛밥', '얼갈이배추국밥', '된장돼지고기덮밥', '닭고기유부밥', '로제소스 리소토 ', '달걀말이밥', '아스파라거스 전복볶음밥', '옥수수김밥', '생와사비 참치마요김밥', '떠먹는 초밥 ', '중화풍 달걀볶음밥', '불고기장 쌈밥', '오므라이스', '캘리포니아롤', '돌솥비빔밥', '에비동', '유산슬덮밥', '불고기김초밥', '토마토소스 해물리소토', '카르보나라 라이스', '쇠고기김치주먹밥', '중화풍 호박고지덮밥', '테이크아웃 비빔밥', '시푸드데리컵밥', '버섯크림리소토', '알리오올리오 라이스', '4색덮밥', '김치볶음밥 돈가스롤', '소보로 오곡유부주머니', '스파이시닭구이 커리', '전복버터볶음밥', '약고추장곰취쌈밥', '데리야키마늘소스 채소덮밥', '매실장아찌볶음밥', '아스파라거스볶음밥', '모시조갯국', '돼지고기짬뽕국', '소고기 뭇국', '전복삼계탕', '백합 맑은탕', '봄동쇠고기국', '닭가슴살완자국', '마늘 조개탕', '한방오리백숙', '맑은 생태탕', '토마토누룽지수프', '갈비탕', '맑은 꽃게탕', '고사리 육개장', '5분 어묵국', '닭곰탕', '감자버섯수제비', '낙지불고기매운탕', '매콤한 쇠고기뭇국', '맑은 쇠고기뭇국', '얼큰홍합탕', '대파닭개장', '매운어묵배춧국', '삼계탕', '부추닭고기 물만두탕', '쇠고기우거지된장국', '경상도식 탕국', '쇠고기부추들깨탕', '감잣국', '개성만둣국', '굴짬뽕만둣국', '얼큰누룽지탕', '크림소스 달래뇨끼', 